<p></p>
<br />
<div id="sec_intro" style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color=#FF7500 size=6>
Data Crawling
        </font>
		<p></p>
		<hr>
        برای مشاهده صحیح این نوت بوک فونت XB Zar را بر روی کامپیوتر خود داشته باشید.
        <br>        
        <br>
        در این نوت‌بوک، داده‌های خبری را از سایت <a href="https://farsi.alarabiya.net">alarabiya</a> استخراج و مرتب سازی می‌کنیم.
    به این منظور از ابزار BeautifulSoup استفاده می‌کنیم که HTML را گرفته و یک متن برمیگرداند. سپس متن به دست آمده را پردازش می‌کنیم.
    به عنوان مثال:
        <br>
        <br>
        <table style="width: 50%; font-size: 15px">
    <thead>
      <tr>
        <th style="text-align:center">تگ موجود در HTML</th>
        <th style="text-align:center">داده استخراج شده</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="text-align:center">h1</td>
        <td style="text-align:center">تیتر خبر</td>
      </tr>
      <tr>
        <td style="text-align:center">p</td>
        <td style="text-align:center">متن خبر</td>
      </tr>
      <tr>
        <td style="text-align:center">a</td>
        <td style="text-align:center">لیبل خبر</td>
      </tr>
    </tbody>
</table>
	</font>
</div>

In [1]:
import csv
import numpy as np
import pandas as pd
import requests
import tqdm
from bs4 import BeautifulSoup

In [5]:
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

In [6]:
news_link = []
def alarabiya_extract_news_link(url,pages):
    for i in tqdm.tqdm(range(pages), leave=True, desc=url+"1:"+str(pages)):
        index = i + 2;
        req = requests.get(url+str(index),timeout=5, headers=headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        a = soup.find_all(attrs={"class": "latest_link"})
        for i in a:
            news_link.append(i['href'])

    return news_link

In [7]:
def alarabiya_remove_invalid_links(news_link):
    for i  in range(len(news_link)):
        if news_link[i].find('https://') == -1:
            news_link[i] = 'https://farsi.alarabiya.net' + news_link[i] 

    new_news_link =[]
    for i  in range(len(news_link)):
        if news_link[i].find('https://farsi.alarabiya.net') != -1:
            new_news_link.append(news_link[i])

    print(len(new_news_link))
    return new_news_link

In [8]:
def alarabiya_extract_news_text(new_news_link):
    news_texts=[]
    labels =[]
    text = []
    newsText = ''
    pbar = tqdm.tqdm(new_news_link, leave=True)
    for url in pbar:
        pbar.set_description(url)
        req = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(req.content, 'html.parser')
        
        title = soup.find_all("h1")
        if title:
            title = (' ').join((title[0].text).split())
        
        a = soup.find_all("p", {"class": "body-1 paragraph"})
        a_len = len(a)
        if a_len != 0:
            for x in range(a_len): 
                newsText = newsText + (' ').join((a[x].text).split())

        tags = soup.find_all("a",{"class":"tags_link section"})
        a_tags = []
        if len(tags) != 0:
            for tag in tags:
                labels.append((' ').join((tag.text).split()))

        if len(newsText) != 0 and len(labels) != 0:
            news_texts.append({"title":title,"text": newsText, "label": labels})

        newsText = ''
        text = []
        labels = []

    return news_texts


<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
       🔷 استخراج داده خام
</font> 
</div>


In [9]:
urls = [
{'url': 'https://farsi.alarabiya.net/iran/archive?pageNo=', 'pages': 100},
{'url': 'https://farsi.alarabiya.net/middle-east/archive?pageNo=', 'pages': 100},
{'url': 'https://farsi.alarabiya.net/international/archive?pageNo=', 'pages': 100},
{'url': 'https://farsi.alarabiya.net/sport/archive?pageNo=', 'pages': 100},
{'url': 'https://farsi.alarabiya.net/afghanistan/archive?pageNo=', 'pages': 100},
{'url': 'https://farsi.alarabiya.net/variety/archive?pageNo=', 'pages': 100}
]

alarabiya_news_link = []
for url in urls:
    a = alarabiya_extract_news_link(url['url'],url['pages'])
    alarabiya_news_link = [*alarabiya_news_link, *a]

https://farsi.alarabiya.net/iran/archive?pageNo=1:100: 100%|██████████| 100/100 [00:41<00:00,  2.40it/s]
https://farsi.alarabiya.net/middle-east/archive?pageNo=1:100: 100%|██████████| 100/100 [00:43<00:00,  2.31it/s]
https://farsi.alarabiya.net/international/archive?pageNo=1:100: 100%|██████████| 100/100 [00:41<00:00,  2.38it/s]
https://farsi.alarabiya.net/sport/archive?pageNo=1:100: 100%|██████████| 100/100 [00:41<00:00,  2.42it/s]
https://farsi.alarabiya.net/afghanistan/archive?pageNo=1:100: 100%|██████████| 100/100 [00:43<00:00,  2.32it/s]
https://farsi.alarabiya.net/variety/archive?pageNo=1:100: 100%|██████████| 100/100 [00:42<00:00,  2.35it/s]


In [10]:
print("Alarabiya News Link Length: ", len(alarabiya_news_link))

Alarabiya News Link Length:  20736


In [ ]:
alarabiya_news_link

['/iran/2023/03/06/قوه-قضائیه-احکام-اعدام-برای-6-زندانی-سیاسی-عرب-را-تایید-کرد',
 '/iran/2023/03/05/سناتور-هاگرتی-با-انتقاد-از-دولت-بایدن-خواستار-فعال-شدن-سازوکار-ماشه-علیه-ایران-شد',
 '/iran/2023/03/05/استقرار-نیروهای-امنیتی-جمهوری-اسلامی-در-مدارس-به-بهانه-مقابله-با-مسمومیت-دانش-آموزان',
 '/iran/2023/03/05/نتانیاهو-سخنان-گروسی-در-غیرقانونی-بودن-حمله-به-تاسیسات-اتمی-ایران-را-بی-ارزش-دانست',
 '/international/2023/03/05/موسسه-علوم-و-امنیت-بین-المللی-ایران-طی-سه-ماه-توانایی-ساخت-7-بمب-هسته-ای-را-دارد',
 '/iran/2023/03/05/تداوم-حملات-شیمیایی-به-مدارس-دخترانه-در-ایران',
 '/iran/2023/03/05/پزشک-متخصص-دسترسی-به-گازهای-به-کاررفته-در-مسمومیت-ها-برای-افراد-عادی-غیرممکن-است',
 '/iran/2023/03/05/کمالوندی-توافق-با-آژانس-برای-دسترسی-به-افراد-و-سایت-های-مشکوک-اعلام-نشده-را-رد-کرد',
 '/international/2023/03/05/آژانس-ایران-از-موضع-خود-درباره-سایت-های-هسته-ای-مشکوک-اعلام-نشده-عقب-نشینی-کرد',
 '/iran/2023/03/04/تداوم-حملات-گسترده-و-هماهنگ-شیمیایی-به-مدارس-دخترانه-در-سراسر-ایران',
 '/iran/2023/03/04/گروسی

<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
       🔷 حذف داده‌های نامعتبر و تکراری از دیتای استخراج شده اولیه
</font> 
</div>

In [11]:
alarabiya_valid_news_link = alarabiya_remove_invalid_links(alarabiya_news_link)


alarabiya_valid_news_link

20736


['https://farsi.alarabiya.net/iran/2023/03/06/واکنش-دیرهنگام-خامنه-ای-سررشته-مسمومیت-های-سریالی-را-پیدا-کنید',
 'https://farsi.alarabiya.net/iran/2023/03/06/تداوم-حملات-شیمیایی-به-مراکز-آموزشی-دخترانه-در-ایران',
 'https://farsi.alarabiya.net/iran/2023/03/06/بررسی-پرونده-هسته-ای-ایران-در-روز-نخست-نشست-شورای-حکام',
 'https://farsi.alarabiya.net/iran/2023/03/06/-قوه-قضائیه-ایران-بازداشت-دست-کم-100-هزار-نفر-در-جریان-اعتراضات-اخیر-را-تایید-کرد',
 'https://farsi.alarabiya.net/middle-east/2023/03/06/پادشاه-اردن-خواستار-حمایت-آمریکا-برای-مقابله-با-شبه-نظامیان-وابسته-به-ایران-شد-',
 'https://farsi.alarabiya.net/international/2023/03/06/فاکس-نیوز-روسیه-موافقت-کرد-تا-ذخایر-اورانیوم-غنی-سازی-شده-ایران-را-بازگرداند',
 'https://farsi.alarabiya.net/iran/2023/03/06/قوه-قضائیه-احکام-اعدام-برای-6-زندانی-سیاسی-عرب-را-تایید-کرد',
 'https://farsi.alarabiya.net/iran/2023/03/05/سناتور-هاگرتی-با-انتقاد-از-دولت-بایدن-خواستار-فعال-شدن-سازوکار-ماشه-علیه-ایران-شد',
 'https://farsi.alarabiya.net/iran/2023/03/05/اس

In [12]:
alarabiya_valid_news_link = np.unique(alarabiya_valid_news_link)


print("Alarabiya Valid_News_Link Length: ", len(alarabiya_valid_news_link))

Alarabiya Valid_News_Link Length:  5188


In [13]:
alarabiya_news_texts = alarabiya_extract_news_text(alarabiya_valid_news_link)

https://farsi.alarabiya.net/variety/2023/02/20/کوسه-شش-متری-سر-ماهیگیر-مکزیکی-را-قطع-کرد: 100%|██████████| 5188/5188 [58:28<00:00,  1.48it/s]


In [14]:
len(alarabiya_news_texts)

4426

<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
       🔷 ذخیره داده‌های پیش‌پردازش شده در فایل
</font> 
</div>

In [15]:
data_file = open('alarabiya_data.csv', 'a', newline='')
csv_writer = csv.writer(data_file)

count = 0
for i in alarabiya_news_texts:
    if count == 0:
        header = i.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(i.values())

data_file.close()


<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
       🔷 نمایی از دیتاست جامع جمع‌آوری شده
        <br>
        حجم فایل: 13 مگابایت
        <br>    
	</font> 
</div>


In [2]:
data = pd.read_csv('alarabiya_data.csv')

data

,title,text,label
0,تصویری؛ 35 عکسی که در زمان مناسب گرفته شده است,عکس‌های متمایز همیشه توسط عکاسان حرفه‌ای گرفته...,"['عکس دقیق', 'عکس به موقع', 'عکاس ماهر عکاس آم..."
1,اگر فوائد پوست و دانه‌های داخل سیب را بدانید، ...,اگر از عاشقان سیب هستید، باید بدانید، هسته داخ...,"['پوست سیب', 'سیب', 'سلامتی برای یدن', 'مصرف ه..."
2,«الساندرا گالونی» اولین سردبیر زن خبرگزاری «رو...,خبرگزاری «رویترز»، روز سه‌شنبه اعلام کرد، السا...,"['گالونی', 'خبرگزاری رویترز', 'سردبیر رویترز',..."
3,با این دستگاه پیشرفته خروپف را کاهش دهید,سازمان غذا و داروی آمریکا، با بازاریابی دستگاه...,"['خروپف کردن', 'سازمان غذا و داروی آمریکا', 'د..."
4,هشدار دانشمندان درباره افزایش حرکت یخچال‌های ط...,بر اساس گزارش پارک ملی و منطقه حفاظت‌شده «دنال...,"['حرکت یخچال\u200cهای طبیعی', 'جونی کینگزلاک، ..."
...,...,...,...
4421,چگونه همه‌گیری کرونا الگوی سینما رفتن را تغییر...,کارشناسان بر این باورند که همه‌گیری کرونا الگو...,"['کرونا', 'سینما', 'اسکار']"
4422,هیو لوری در فصل سوم سریال «تهران» نقش‌آفرینی م...,«اپل تی‌وی پلاس» اعلام کرد قصد دارد فصل سوم سر...,"['اسرائیلی', 'ایران', 'تهران', 'سریال']"
4423,کارهای جالبی که موتور جستجوی جدید «بینگ» مایکر...,شرکت «مایکروسافت» اخیرا در رقابت با «گوگل» روی...,"['مایکروسافت', 'بینگ']"
4424,احیای ماموت‌ پشمالو برای مقابله با تغییرات اقلیمی,دانشمندان قصد دارند ماموت پشمالو که حدود 4000 ...,"['سیبری', 'ماموت پشمالو']"



<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
       🔷 تهیه دیتاست کم حجم (حداکثر 5 مگابایت) از دیتاست جامع اولیه
        <br>
	</font> 
</div>

In [4]:
data_file = open('alarabiya_dataset.csv', 'w', newline='')
csv_writer = csv.writer(data_file)

count = 0

for i in alarabiya_news_texts[0:1500]:
    if count == 0:
        header = i.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(i.values())

data_file.close()

NameError: name 'alarabiya_news_texts' is not defined


<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
       🔷 نمایی از دیتاست کم حجم تهیه شده
        <br><br>
        حجم فایل: 4.8 مگابایت
        <br>
	</font> 
</div>

In [21]:
data = pd.read_csv('alarabiya_dataset.csv')

data

,title,text,label
0,تصویری؛ 35 عکسی که در زمان مناسب گرفته شده است,عکس‌های متمایز همیشه توسط عکاسان حرفه‌ای گرفته...,"['عکس دقیق', 'عکس به موقع', 'عکاس ماهر عکاس آم..."
1,اگر فوائد پوست و دانه‌های داخل سیب را بدانید، ...,اگر از عاشقان سیب هستید، باید بدانید، هسته داخ...,"['پوست سیب', 'سیب', 'سلامتی برای یدن', 'مصرف ه..."
2,«الساندرا گالونی» اولین سردبیر زن خبرگزاری «رو...,خبرگزاری «رویترز»، روز سه‌شنبه اعلام کرد، السا...,"['گالونی', 'خبرگزاری رویترز', 'سردبیر رویترز',..."
3,با این دستگاه پیشرفته خروپف را کاهش دهید,سازمان غذا و داروی آمریکا، با بازاریابی دستگاه...,"['خروپف کردن', 'سازمان غذا و داروی آمریکا', 'د..."
4,هشدار دانشمندان درباره افزایش حرکت یخچال‌های ط...,بر اساس گزارش پارک ملی و منطقه حفاظت‌شده «دنال...,"['حرکت یخچال\u200cهای طبیعی', 'جونی کینگزلاک، ..."
...,...,...,...
445,لاوروف: روسیه مخالف حضور نظامی آمریکا و ناتو د...,سرگئی لاوروف، وزیر خارجه روسیه در نشست دو روزه...,"['طالبان', 'داعش', 'روسیه', 'افغانستان']"
446,روسیه اولین دیپلمات حکومت طالبان را پذیرفت,سرگی لاوروف، وزیر خارجه روسیه از پذیرش اولین د...,"['طالبان', 'دیپلمات', 'لاوروف', 'به رسمیت شناخ..."
447,طالبان و نیروهای «جبهه مقاومت ملی» افغانستان د...,جبهه مقاومت ملی افغانستان به رهبری احمد مسعود ...,"['پنجشیر', 'طالبان', 'افغانستان', 'احمد مسعود']"
448,شورای حقوق بشر سازمان ملل برای افغانستان گزارش...,با افزایش گزارش‌ها مبنی بر موارد نقض حقوق بشر ...,"['آنتونیو گوترش', 'طالبان', 'ریچارد بنت', 'افغ..."
